In [2]:
import requests
import pandas as pd
import yaml
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

In [3]:
with open('../config/config.yaml', 'r') as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)

unidad = 1e6
mdp = 1e6
mmdp = 1e6*1000

| Id Serie | Descripcion | url  |
|----------|-------------|------|
| SM13     | Circulación de moneda metálica de 1 pesos | https://www.banxico.org.mx/SieAPIRest/service/v1/series/SM13    |
| SM14     | Circulación de moneda metálica de 2 pesos | https://www.banxico.org.mx/SieAPIRest/service/v1/series/SM14    |
| SM15     | Circulación de moneda metálica de 5 pesos | https://www.banxico.org.mx/SieAPIRest/service/v1/series/SM15    |
| SM16     | Circulación de moneda metálica de 10 pesos| https://www.banxico.org.mx/SieAPIRest/service/v1/series/SM16    |
| SM1472   | Total de Circulación de billete 20 pesos  | https://www.banxico.org.mx/SieAPIRest/service/v1/series/SM1472  |
| SM1478   | Total de Circulación de billete 50 pesos  | https://www.banxico.org.mx/SieAPIRest/service/v1/series/SM1478  |
| SM1479   | Total de Circulación de billete 100 pesos | https://www.banxico.org.mx/SieAPIRest/service/v1/series/SM1479  |
| SM1480   | Total de Circulación de billete 200 pesos | https://www.banxico.org.mx/SieAPIRest/service/v1/series/SM1480  |
| SM1481   | Total de Circulación de billete 500 pesos | https://www.banxico.org.mx/SieAPIRest/service/v1/series/SM1481  |
| SM1482   | Total de Circulación de billete 1000 pesos | https://www.banxico.org.mx/SieAPIRest/service/v1/series/SM1482  |
|          |                                            |                                                                 |
|SP1       | IPC Por objeto del gasto Nacional Indice General | https://www.banxico.org.mx/SieAPIRest/service/v1/series/SP1

In [4]:
infoConsultasBilletes = {
    "SM1472": "https://www.banxico.org.mx/SieAPIRest/service/v1/series/SM1472",
    "SM1478": "https://www.banxico.org.mx/SieAPIRest/service/v1/series/SM1478",
    "SM1479": "https://www.banxico.org.mx/SieAPIRest/service/v1/series/SM1479",
    "SM1480": "https://www.banxico.org.mx/SieAPIRest/service/v1/series/SM1480",
    "SM1481": "https://www.banxico.org.mx/SieAPIRest/service/v1/series/SM1481",
    "SM1482": "https://www.banxico.org.mx/SieAPIRest/service/v1/series/SM1482"
    }
infoConsultasMonedas = {
    "SM13": "https://www.banxico.org.mx/SieAPIRest/service/v1/series/SM13",
    "SM14": "https://www.banxico.org.mx/SieAPIRest/service/v1/series/SM14",
    "SM15": "https://www.banxico.org.mx/SieAPIRest/service/v1/series/SM15",
    "SM16": "https://www.banxico.org.mx/SieAPIRest/service/v1/series/SM16"}

In [5]:
def consultaTotalBilletesMonedas(key: str, infoConsultas:dict, config:dict):
    consulta = infoConsultas[key]
    consultarHistoricos = "/datos/?"
    use_token = f"token={config['BM_token']}"
    url = consulta + consultarHistoricos + use_token
    try:
        respuesta = requests.get(url).json()
        title = respuesta["bmx"]["series"][0]["titulo"]
        data = respuesta["bmx"]["series"][0]["datos"]
        df_query = pd.DataFrame(data)
        df_query.fecha  = pd.to_datetime(df_query.fecha, format="%d/%m/%Y")
        df_query = df_query.set_index("fecha")
        df_query["dato"] = df_query["dato"].str.replace(',', '', regex=False)
        df_query['dato'] = df_query['dato'].astype(float)
        df_query = df_query.rename(columns={"dato": " ".join(title.split(" ")[-2:]) })
        return title, df_query
    except Exception as e:
        None
        return "Inconsistencia al obtener la consulta", pd.DataFrame()

In [6]:
consulta = "https://www.banxico.org.mx/SieAPIRest/service/v1/series/SP1"

consultarHistoricos = "/datos/?"
use_token = f"token={config['BM_token']}"
url = consulta + consultarHistoricos + use_token

respuesta = requests.get(url=url).json()
title = respuesta["bmx"]["series"][0]["titulo"]
data = respuesta["bmx"]["series"][0]["datos"]
df_query = pd.DataFrame(data)
df_query.fecha  = pd.to_datetime(df_query.fecha, format="%d/%m/%Y")
df_query = df_query.set_index("fecha")
df_query.shift(periods=1)

,dato
fecha,
1969-01-01,None
1969-02-01,0.011595000000
1969-03-01,0.011636000000
1969-04-01,0.011648000000
1969-05-01,0.011680000000
...,...
2024-11-01,136.828000000000
2024-12-01,137.424000000000
2025-01-01,137.949000000000


In [7]:
dfBilletes = pd.DataFrame()
dfMonedas = pd.DataFrame()
Valores = dict()
for key in infoConsultasBilletes.keys():
    title, df_query = consultaTotalBilletesMonedas(key=key,
                                        infoConsultas=infoConsultasBilletes,
                                        config=config)
    clave = " ".join(title.split(" ")[-2:])
    valor = int(title.split(" ")[-2].replace(',', ''))
    Valores[clave] = valor
    dfBilletes = pd.merge(left=dfBilletes,
                  right=df_query,
                  left_index=True,
                  right_index=True,
                  how="outer")
    

for key in infoConsultasMonedas.keys():
    title, df_query = consultaTotalBilletesMonedas(key=key,
                                        infoConsultas=infoConsultasMonedas,
                                        config=config)
    clave = " ".join(title.split(" ")[-2:])
    valor = int(title.split(" ")[-2].replace(',', ''))
    Valores[clave] = valor
    dfMonedas = pd.merge(left=dfMonedas,
                  right=df_query,
                  left_index=True,
                  right_index=True,
                  how="outer")

In [8]:
fig = fig = go.Figure()

for col in dfBilletes.columns:
    fig.add_trace(go.Scatter(x=dfBilletes.index, 
                             y=dfBilletes[col],
                             name= col,
                             marker_symbol="star"))

fig.update_layout(title_text="Total de billetes en circulación",
                  yaxis_title='Millones de piezas')
fig.show()



fig = fig = go.Figure()

for col in dfMonedas.columns:
    fig.add_trace(go.Scatter(x=dfMonedas.index, 
                             y=dfMonedas[col],
                             name= col,
                             marker_symbol="star"))

fig.update_layout(title_text="Total de Monedas en circulación",
                  yaxis_title='Millones de piezas')

fig.show()

In [9]:
df = pd.merge(left=dfMonedas,
         right=dfBilletes,
         left_index=True,
         right_index=True,
         how="outer")

df = df[df.index >= pd.to_datetime("2020-01-02")]

ultima_fila = df.iloc[-1]
treemap_df = ultima_fila.reset_index()
treemap_df.columns = ['Denominacion', 'Cantidad (Millones de Piezas)']

def obtenerValor(valor_str):
    numero = int(valor_str.split()[0].replace(',', ''))
    return numero
treemap_df['Valor'] = treemap_df['Denominacion'].apply(obtenerValor)

def clasificar(valor_str):
    # Extrae el número del string (asume que el número está al inicio)
    numero = int(valor_str.split()[0].replace(',', ''))
    return 'moneda' if numero < 20 else 'billete'

# Aplicar la función
treemap_df['Formato'] = treemap_df['Denominacion'].apply(clasificar)
fig = px.treemap(
    treemap_df,
    path=['Formato', "Denominacion"],
    values='Cantidad (Millones de Piezas)',
    title='Total de billetes y monedas',
    color_continuous_scale="Blues"
)

fig.show()

In [10]:
treemap_df["Total"] = treemap_df["Cantidad (Millones de Piezas)"]*unidad*treemap_df["Valor"]/mdp
fig = px.treemap(
    treemap_df,
    path=["Formato", "Denominacion"],
    values='Total',
    title='Total de dinero en efectivo en circulación (MDP) al {} existen {} MDP'.format(df.index.max().strftime("%d-%m-%Y"), format(round(treemap_df.Total.sum(),2), ",")),
    color_continuous_scale="Blues"
)
fig.update_traces(
    texttemplate='%{label}<br>%{value:,.0f}<br>%{percentParent:.1%}',
    textfont_size=11
)

fig.show()

In [11]:
ultima_fila = df.iloc[-1]
treemap_df = ultima_fila.reset_index()
treemap_df.columns = ['Denominacion', 'Cantidad (Millones de Piezas)']
# Creamos el treemap
fig = px.treemap(
    treemap_df,
    path=['Denominacion'],
    values='Cantidad (Millones de Piezas)',
    title='Total de billetes',
    color_continuous_scale="Blues"
)

fig.show()

In [12]:
# Convertimos el diccionario en una Serie para alinearlo con columnas
multiplicadores_series = pd.Series(Valores)
# Multiplicamos cada fila por los valores del diccionario y sumamos
df['Total'] = (df*unidad).mul(multiplicadores_series).sum(axis=1)


In [13]:

fig = fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, 
                             y=df.Total/mmdp,
                             name= col,
                             marker_symbol="star"))

fig.update_layout(title_text="Total del flujo en efectivo",
                  yaxis_title='Miles de MDP')

fig.show()

In [43]:
inflacionHistorica = {
    "Mensual": f" https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/910399/es/0700/false/BIE/2.0/{config['INEGI_token']}?type=json",
    "Quincenal": f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/910399,910427/es/0700/false/BIE/2.0/{config['INEGI_token']}?type=json"
}

In [68]:
def inflacion(inflacionHistorica:dict, 
              Periodo:str,
              min_date = str):
    InflacionHistoricaQ = inflacionHistorica[Periodo]
    respuesta = requests.get(url=InflacionHistoricaQ).json()
    data = respuesta["Series"][0]["OBSERVATIONS"]
    df_inflacion = pd.DataFrame(data=data)
    df_inflacion["TIME_PERIOD"]  = pd.to_datetime(df_inflacion["TIME_PERIOD"], format="%Y/%m")
    df_inflacion["OBS_VALUE"] = df_inflacion["OBS_VALUE"].apply(lambda x: float(x))
    df_inflacionPeriodo = df_inflacion.set_index("TIME_PERIOD")[["OBS_VALUE"]].sort_index().rename(columns={"OBS_VALUE": f"Inflacion_{Periodo}"})
    df_inflacionPeriodo = df_inflacionPeriodo[df_inflacionPeriodo.index >= pd.to_datetime(min_date, format = "%d-%m-%Y")]
    return 

In [ ]:
df_inflacion = inflacion(inflacionHistorica=inflacionHistorica, 
                         Periodo="Mensual", 
                         min_date="29-06-1998")

ValueError: could not convert string to float: ''

In [ ]:
# df_inflacion_acumulada = (1+df_inflacion["Inflacion_Mensual"].cumsum()) -1
# df_inflacion_acumulada
df_inflacion["Inflacion_Mensual"]

TIME_PERIOD
1998-07-01    0.96
1998-08-01    0.96
1998-09-01    1.62
1998-10-01    1.43
1998-11-01    1.77
              ... 
2024-11-01    0.44
2024-12-01    0.38
2025-01-01    0.29
2025-02-01    0.28
2025-03-01    0.31
Name: Inflacion_Mensual, Length: 321, dtype: float64

In [ ]:
# PoblacionTotal = f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/1002000001/es/0700/false/BISE/2.0/{config['INEGI_token']}?type=json"
# respuesta = requests.get(url=PoblacionTotal).json()

[(k,v) for k,v in respuesta["Series"][0].items()]
a = respuesta["Series"][0].items()

(respuesta["Series"][0])

{'INDICADOR': '1002000001',
 'FREQ': '7',
 'TOPIC': '123',
 'UNIT': '188',
 'UNIT_MULT': '',
 'NOTE': '1398',
 'SOURCE': '2,3,343,487,781,1668,1669,1670,1671,1672,1677,1678,1679,1714,3001',
 'LASTUPDATE': '21/10/2024 12:00:00 a. m.',
 'STATUS': None,
 'OBSERVATIONS': [{'TIME_PERIOD': '1910',
   'OBS_VALUE': '15160369.00000000000000000000',
   'OBS_EXCEPTION': None,
   'OBS_STATUS': '3',
   'OBS_SOURCE': '',
   'OBS_NOTE': '',
   'COBER_GEO': '0700'},
  {'TIME_PERIOD': '1921',
   'OBS_VALUE': '14334780.00000000000000000000',
   'OBS_EXCEPTION': None,
   'OBS_STATUS': '3',
   'OBS_SOURCE': '',
   'OBS_NOTE': '',
   'COBER_GEO': '0700'},
  {'TIME_PERIOD': '1930',
   'OBS_VALUE': '16552722.00000000000000000000',
   'OBS_EXCEPTION': None,
   'OBS_STATUS': '3',
   'OBS_SOURCE': '',
   'OBS_NOTE': '',
   'COBER_GEO': '0700'},
  {'TIME_PERIOD': '1940',
   'OBS_VALUE': '19653552.00000000000000000000',
   'OBS_EXCEPTION': None,
   'OBS_STATUS': '3',
   'OBS_SOURCE': '',
   'OBS_NOTE': '',
   'CO